# Configuration


## Usage
Let's assume you started a new project and created a [`jupyter notebook`](https://jupyter.org/)

In [ ]:
%ls

### Initialize a new project

The recommended way is to import the `CONFIG` and initialize a new project. However this isn't required just recommended.

In [ ]:
from pyglotaran_extras import CONFIG

CONFIG.init_project()

This will look up all config files in your home folder, the notebook folders parent folder, 
and the notebook folder, combine them and create a new config and schema file in the notebook 
folder for you, as well as rediscovering and reloading the config (see [file-lookup](#file-lookup)).

```{note}
If a config file already exists, the file creation will be skipped in order to not overwrite an
exported custom schema with your own plot functions.
```


```{admonition} Tip
If you don't want the config to be shown in the cell output, just add a `;` after `CONFIG.init_project()`.
```

In [ ]:
%ls

### Discovering and loading config files

If you want to only work with one config file you can simply load it.

In [ ]:
CONFIG.load("../fs_config.yml")

If you don't like the way config files are looked up you can manually rediscover them and reload the config.

```{note}
Note that the reload is only used for demonstration purposes, since the config is autoreloaded before being used (see [auto-reload](#auto-reload))
```

In [ ]:
CONFIG.rediscover(include_home_dir=False, lookup_depth=3)
CONFIG.reload()

### How the config effects plotting

To demonstrate the difference between not using the config and using the config we create a copy of our `project_config` as well as an `empty_config` (same as not having a config at all).

In [ ]:
from pyglotaran_extras.config.config import Config

project_config = CONFIG.model_copy()
empty_config = Config()

#### Default plotting behavior

In [ ]:
from glotaran.testing.simulated_data.parallel_spectral_decay import DATASET

from pyglotaran_extras import plot_data_overview

CONFIG._reset(empty_config)

plot_data_overview(DATASET);

#### Manually adjusting the Plot

In [ ]:
fig, axes = plot_data_overview(DATASET, linlog=True, linthresh=2, use_svd_number=True)
axes[0].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("Time (ps)")
axes[1].set_xlabel("Time (ps)")
axes[1].set_ylabel("")
axes[2].set_ylabel("Singular Value (a.u.)")
axes[3].set_xlabel("Wavelength (nm)")
axes[3].set_ylabel("");

#### Using the plot config

In [ ]:
CONFIG._reset(project_config)

plot_data_overview(DATASET);

## FAQ
### Do I have to use the config?
No. Using the config is fully optional, however we recommend using it since it reduces the amount of 
code you need to write and lets anybody reading your analysis focus on the science rather than the 
python code used to make your plots.

### What can the configuration be used for?
The main goal of the config is to configure plot functions and reduce tedious code duplication like:
- Renaming labels of axes
- Overriding default values to plot function calls

We try to have sensible default values for our plot functions, but there is no `one fits all` solution.

Especially since arguments like `linthresh` (determines the range in which a `linlog` plot is linear)
are highly dependent on your data.

Thus we give you the power to customize the default values to your projects needs, without having 
repeating them over and over each time you call a plot function.


### Can I still change plot labels myself?
Yes, the config gets applied when a config enabled plot function is called you can still 
work with the return figure and axes as you are used to be.


### Does using a config mean arguments I pass to a function get ignored?
No, arguments from the config are only used you don't pass an argument.

(file-lookup)=
### How are config files looked up?
When you import anything from `pyglotaran_extras` the location of your project is determined.
This location then is used to look for `pygta_config.yaml` and `pygta_config.yml` in the following folders:
- Your user home directory
- The projects parent folder
- The project folder

If you don't want to include your home folder or a different lookup depth relative to your project 
folder you can use `CONFIG.rediscover`.
If you only want to load the config from a single file you can use `CONFIG.load`.

(auto-reload)=
### Do I need to reload the config after changing a file?
No, the config keeps track of when each config file was last modified and automatically reloads if needed.

### How is determined what config values to use?
The config follows the locality and specificity principles.

#### Locality
Locality here means that the closer the configuration is to the plot function call the higher its importance.

Lets consider the example of the default behavior where configs are looked up in the home directory,
projects parent folder and project folder.
When the global ``CONFIG`` instance is loaded it merges the configs in the following order:
- Your user home directory
- The projects parent folder
- The project folder

Where each merge overwrites duplicate values from the config it gets merged into.

#### Specificity
For ease of use and reduced duplications, the plot config has a ``general`` section 
that applies to a plot function with use those arguments or plot labels.

Lets assume that your experimental data use time in picoseconds (ps) and wavelengths in nanometers (nm).
Instead of a defining the label override for each function you can simply it to the general section as
see above and if a function doesn't have it defined itself it also gets applied for this function.

In [ ]:
CONFIG.plotting.get_function_config("plot_svd")

Now let's assume that one dataset uses wavenumbers instead of wavelength as spectral axis.

You can simply define a `PerFunctionPlotConfig` and call your plot function inside of a `plot_config_context`.
This way you can even override function specific configuration defined in your config file.

In [ ]:
from pyglotaran_extras import PerFunctionPlotConfig, plot_config_context

wavenumber_config = PerFunctionPlotConfig(
    axis_label_override={"spectral": "Wavenumber (cm$^{-1}$)"}
)

with plot_config_context(wavenumber_config):
    plot_svd_config_wavenumber = CONFIG.plotting.get_function_config("plot_svd")
plot_svd_config_wavenumber

This change is only valid inside of the `plot_config_context` and reset afterwards

Which arguments and label are used are defined by the following hierarchy.
- Plot function arguments
- ``plot_config_context``
- Global ``CONFIG.plotting`` instance ``function config``
- Global ``CONFIG.plotting`` instance ``general``

````{note}
For compound functions like `plot_overview` which consist of multiple plot config enabled functions
the `default_args_override` for `plot_overview` will be passed down to the other functions and 
override their usage of own `default_args_override` config (if arguments are passed they aren't 
default arguments anymore 😅).
Where as `axis_label_override` for the functions config is first applied to the intermediate plots 
and `axis_label_override` from `plot_overview` is only applied after that on final plot.

```mermaid
graph TD
    A[plot_overview] --> |"default_args_override (plot_overview)"| B[plot_svd]
    B --> |"axis_label_override (plot_svd)"| C[intermediate plot]
    C --> |"axis_label_override (plot_overview)"| D[final plot]
```
````

In [ ]:
CONFIG.plotting.get_function_config("plot_svd")